# Comparative Acquisition of Complex Sentence Structures in Bilingual and Monolingual Children

## 1. Load all datasets

Miami English dataset contains frog story narratives collected in Miami, Florida. Kids 2nd graders (7/8yo) and 5th graders(7/8yo)

1. **English-MiamiBiling/**: Bilingual English data
2. **English-MiamiMono/**: Monolingual English data
3. **Spanish-MiamiBiling/**: Bilingual Spanish data

In [4]:
import pylangacq as pla
import os
import glob
import pandas as pd

In [5]:
def robust_load_cha(file_path):
    """
    Robust function to load .cha files with fallback strategies
    Returns: (success, chat_object_or_utterance_count, method_used)
    """
    
    # Strategy 1: Try normal pylangacq loading
    try:
        chat = pla.read_chat(file_path)
        return True, chat, 'pylangacq'
    except Exception as e1:
        # Strategy 2: Try with explicit encoding
        try:
            chat = pla.read_chat(file_path, encoding='utf-8')
            return True, chat, 'pylangacq_utf8'
        except Exception as e2:
            # Strategy 3: Raw parsing for basic utterance count
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                lines = content.split('\n')
                chi_utterances = [line for line in lines if line.strip().startswith('*CHI:')]
                return False, chi_utterances, 'raw_parsing'
            except Exception as e3:
                return False, 0, 'failed'

def load_directory_structure():
    """
    Comprehensively load all .cha files from all directories
    """
    
    # Define all dataset directories
    datasets = {
        'English-MiamiBiling': {
            'bleng2-audio': 'English_Bilingual_Grade2_Audio',
            'bleng2-noaudio': 'English_Bilingual_Grade2_NoAudio', 
            'bleng5-audio': 'English_Bilingual_Grade5_Audio',
            'bleng5-noaudio': 'English_Bilingual_Grade5_NoAudio'
        },
        'English-MiamiMono': {
            'mleng2': 'English_Monolingual_Grade2',
            'mleng5': 'English_Monolingual_Grade5'
        }
        # ,
        # 'Spanish-MiamiBiling': {
        #     'blspan2-audio': 'Spanish_Bilingual_Grade2_Audio',
        #     'blspan2-noaudio': 'Spanish_Bilingual_Grade2_NoAudio',
        #     'blspan5-audio': 'Spanish_Bilingual_Grade5_Audio', 
        #     'blspan5-noaudio': 'Spanish_Bilingual_Grade5_NoAudio'
        # },
    }
    
    all_datasets = {}
    summary_stats = []
    
    print("Loading all datasets...")
    print("=" * 60)
    
    for main_dir, subdirs in datasets.items():
        for subdir, dataset_name in subdirs.items():
            dir_path = os.path.join(main_dir, subdir)
            
            if not os.path.exists(dir_path):
                print(f"⚠️  Directory not found: {dir_path}")
                continue
                
            # Find all .cha files
            cha_files = glob.glob(os.path.join(dir_path, "*.cha"))
            
            if not cha_files:
                print(f"📁 {dataset_name}: No .cha files found")
                continue
            
            # Load files
            dataset_files = []
            successful_loads = 0
            partial_loads = 0
            failed_loads = 0
            
            for file_path in cha_files:
                success, result, method = robust_load_cha(file_path)
                
                file_info = {
                    'filename': os.path.basename(file_path),
                    'filepath': file_path,
                    'dataset': dataset_name,
                    'main_category': main_dir,
                    'subcategory': subdir,
                    'success': success,
                    'method': method
                }
                
                if success:
                    file_info['chat_object'] = result
                    file_info['participants'] = result.participants()
                    file_info['utterance_count'] = len(result.utterances())
                    successful_loads += 1
                elif method == 'raw_parsing':
                    file_info['utterance_count'] = len(result)
                    file_info['chi_utterances'] = result
                    partial_loads += 1
                else:
                    failed_loads += 1
                
                dataset_files.append(file_info)
            
            # Store dataset
            all_datasets[dataset_name] = dataset_files
            
            # Print summary
            total_files = len(cha_files)
            print(f"📊 {dataset_name}")
            print(f"   Total files: {total_files}")
            print(f"   ✅ Fully loaded: {successful_loads}")
            print(f"   🔶 Partial (raw): {partial_loads}")
            print(f"   ❌ Failed: {failed_loads}")
            print()
            
            # Add to summary stats
            summary_stats.append({
                'dataset': dataset_name,
                'main_category': main_dir,
                'subcategory': subdir,
                'total_files': total_files,
                'successful': successful_loads,
                'partial': partial_loads,
                'failed': failed_loads,
                'success_rate': (successful_loads / total_files * 100) if total_files > 0 else 0
            })
    
    return all_datasets, pd.DataFrame(summary_stats)

In [6]:
all_data, summary_df = load_directory_structure()

Loading all datasets...
📊 English_Bilingual_Grade2_Audio
   Total files: 21
   ✅ Fully loaded: 12
   🔶 Partial (raw): 9
   ❌ Failed: 0

📊 English_Bilingual_Grade2_NoAudio
   Total files: 66
   ✅ Fully loaded: 33
   🔶 Partial (raw): 33
   ❌ Failed: 0

📊 English_Bilingual_Grade5_Audio
   Total files: 16
   ✅ Fully loaded: 7
   🔶 Partial (raw): 9
   ❌ Failed: 0

📊 English_Bilingual_Grade5_NoAudio
   Total files: 73
   ✅ Fully loaded: 47
   🔶 Partial (raw): 26
   ❌ Failed: 0

📊 English_Monolingual_Grade2
   Total files: 43
   ✅ Fully loaded: 19
   🔶 Partial (raw): 24
   ❌ Failed: 0

📊 English_Monolingual_Grade5
   Total files: 46
   ✅ Fully loaded: 29
   🔶 Partial (raw): 17
   ❌ Failed: 0



In [7]:
# Display comprehensive summary
print("\n" + "="*80)
print("📈 COMPREHENSIVE DATASET SUMMARY")
print("="*80)

# Overall statistics
total_datasets = len(all_data)
total_files = sum(len(files) for files in all_data.values())
total_successful = summary_df['successful'].sum()
total_partial = summary_df['partial'].sum() 
total_failed = summary_df['failed'].sum()

print(f"📁 Total datasets: {total_datasets}")
print(f"📄 Total files: {total_files}")
print(f"✅ Fully loaded: {total_successful} ({total_successful/total_files*100:.1f}%)")
print(f"🔶 Partially loaded: {total_partial} ({total_partial/total_files*100:.1f}%)")
print(f"❌ Failed: {total_failed} ({total_failed/total_files*100:.1f}%)")
print(f"📊 Overall success rate: {(total_successful+total_partial)/total_files*100:.1f}%")

print("\n" + "="*80)
print("📋 DETAILED BREAKDOWN BY DATASET")
print("="*80)

# Sort by main category for better organization
summary_df_sorted = summary_df.sort_values(['main_category', 'subcategory'])
print(summary_df_sorted.to_string(index=False))

print("\n" + "="*80)
print("🔍 DATASET ACCESS EXAMPLES")
print("="*80)

# Show examples of how to access data
for dataset_name, files in list(all_data.items())[:3]:  # Show first 3 datasets
    print(f"\n📚 {dataset_name}:")
    print(f"   Access: all_data['{dataset_name}']")
    print(f"   Files: {len(files)}")
    
    if files:
        # Show example file
        example_file = files[0]
        print(f"   Example file: {example_file['filename']}")
        if example_file['success']:
            print(f"   Participants: {example_file['participants']}")
            print(f"   Utterances: {example_file['utterance_count']}")
        else:
            print(f"   Utterances (raw): {example_file.get('utterance_count', 'N/A')}")


📈 COMPREHENSIVE DATASET SUMMARY
📁 Total datasets: 6
📄 Total files: 265
✅ Fully loaded: 147 (55.5%)
🔶 Partially loaded: 118 (44.5%)
❌ Failed: 0 (0.0%)
📊 Overall success rate: 100.0%

📋 DETAILED BREAKDOWN BY DATASET
                         dataset       main_category    subcategory  total_files  successful  partial  failed  success_rate
  English_Bilingual_Grade2_Audio English-MiamiBiling   bleng2-audio           21          12        9       0     57.142857
English_Bilingual_Grade2_NoAudio English-MiamiBiling bleng2-noaudio           66          33       33       0     50.000000
  English_Bilingual_Grade5_Audio English-MiamiBiling   bleng5-audio           16           7        9       0     43.750000
English_Bilingual_Grade5_NoAudio English-MiamiBiling bleng5-noaudio           73          47       26       0     64.383562
      English_Monolingual_Grade2   English-MiamiMono         mleng2           43          19       24       0     44.186047
      English_Monolingual_Grade5   Englis

# 2. Save relevant data to file

In [39]:
from tqdm import tqdm
import pandas as pd

def build_utterances_dataframe(all_data):
    rows = []

    for dataset in all_data.keys():
        for file in tqdm(all_data[dataset], desc=f"Processing {dataset}"):
            if file.get('success') is True:
                plain_utterances = [
                    ' '.join(token.word for token in ut.tokens)
                    for ut in file['chat_object'].utterances()
                ]

                rows.append({
                    'dataset': dataset,
                    'filename': file['filename'],
                    'age': file['chat_object'].ages(),
                    'languages': file['chat_object'].languages(),
                    'is_bilingual': True if 'Bilingual' in dataset else False,
                    'plain_utterances': plain_utterances
                })

    df = pd.DataFrame(rows, columns=['dataset', 'filename', 'age', 'languages', 'is_bilingual', 'plain_utterances'])
    return df

In [40]:
all_utterances_df = build_utterances_dataframe(all_data)
all_utterances_df.to_csv('all_utterances.csv', index=False, encoding='utf-8')

Processing English_Monolingual_Grade5: 100%|██████████| 46/46 [00:00<00:00, 13256.70it/s]
